In [9]:
from MNE_Pipeline_ext import MNE_Repo_Mat_ext
from MNE_Pipeline import MNE_Repo_Mat
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle

In [2]:
mp = MNE_Repo_Mat()
mp_ext = MNE_Repo_Mat_ext()
info = mp.construct_info()

/home/kashraf/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/home/kashraf/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/kashraf/cp_project/MNE_Pipeline.py:53: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read

In [14]:
def arrange_event_wise_RTs(RT, trigs):
    RT_dict = dict()
    event_ids = np.unique(trigs)
    for event in event_ids:
        indices = [i for i in range(len(trigs)) if trigs[i] == event]
        RT_dict[str(event)] = RT[indices]
        
    return RT_dict

In [3]:
subjects = sorted([os.path.splitext(file)[0] for file in os.listdir('Data/') if file.endswith('.mat')])
subjects_path = sorted(['Data/{}'.format(file) for file in os.listdir('Data/') if file.endswith('.mat')])
subject_epoch_paths = [os.path.join('combined_epochs', subject + '_epoch' + '.fif') for subject in subjects]
subject_bts_erp_path = [os.path.join('bootstrap_erps_cl_vs_amb', subject) for subject in subjects]

In [17]:
RT_dict_sub = dict()
for subject, subject_path in tqdm(zip(subjects, subjects_path)):
    data = mp.load_data_mat(subject_path)
    RT = np.nan_to_num(data['RT'], nan=0)
    trigs = data['trigs']
    RT_dict_sub[subject] = arrange_event_wise_RTs(RT, trigs)

50it [00:49,  1.00it/s]


In [4]:
for path, subject in tqdm(zip(subject_epoch_paths, subjects)):
    epochs = mp.load_epochs(path)
    mp_ext.async_bootstrap_epochs(subject, epochs, iterations=500, sampling_rate=30)

0it [00:00, ?it/s]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/N1_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
1it [00:07,  7.28s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/N10_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
2it [00:15,  7.57s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/N11_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
3it [00:22,  7.52s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (c

28it [03:41,  8.34s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/S21_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
29it [03:49,  8.27s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/S3_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
30it [03:57,  8.25s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This filename (combined_epochs/S4_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
31it [04:05,  8.22s/it]/home/kashraf/cp_project/MNE_Pipeline.py:79: RuntimeWarning: This file

In [30]:
with open('bootstrap_erps_cl_vs_amb/T4', 'rb') as file:
    di = pickle.load(file)
len(di['15']) + len(di['3'])

1000

In [5]:
avg_bp_dict = dict()
mp.Fs = 500
for subject, path in tqdm(zip(subjects, subject_bts_erp_path)):
    file = open(path, 'rb')
    data = pickle.load(file)
    mp.epochs_raw = None
    mp.epochs_raw = np.append(data['3'], data['15'], axis=0)
    avg_bp_dict[subject] = mp.get_avg_band_power()
    file.close()

50it [05:38,  6.76s/it]


In [6]:
with open('Avg_bp_BTS_dict_1', 'wb') as f:
    pickle.dump(avg_bp_dict, f)

In [4]:
with open('Avg_bp_BTS_dict_1', 'rb') as f:
    avg_bp_dict = pickle.load(f)

In [5]:
for subject in tqdm(subjects):
    avg_bp = avg_bp_dict[subject]
    mp.async_save_band_power_topo(subject, avg_bp)

100%|██████████| 50/50 [2:08:17<00:00, 153.95s/it]  


In [6]:
for subject in tqdm(subjects):
    avg_bp = avg_bp_dict[subject]
    mp.async_save_combined_topomap(subject, len(avg_bp))

100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


In [7]:
label_mappers = dict()
for subject, path in tqdm(zip(subjects, subject_bts_erp_path)):
    file = open(path, 'rb')
    data = pickle.load(file)
    label_3 = np.full(len(data['3']), 3)
    label_15 = np.full(len(data['15']), 15)
    labels = np.append(label_3, label_15)
    label_mappers[subject] = labels

50it [00:08,  5.61it/s]


In [ ]:
mp.train_test_spliter_ML(subjects, label_mappers, [3,15], 'band_power_topomap_BTS', save_path='data_bts_1')

  0%|          | 0/50 [00:00<?, ?it/s]

200
100


  2%|▏         | 1/50 [00:01<01:13,  1.49s/it]

200
100


  4%|▍         | 2/50 [00:02<01:11,  1.49s/it]

...

200
100


 12%|█▏        | 6/50 [00:08<01:04,  1.47s/it]

200
100


 14%|█▍        | 7/50 [00:10<01:04,  1.50s/it]

200
100


In [11]:
len(os.listdir('data_bts/validation/15'))

2500